In [1]:
from src.arguments import ModelArguments, DataArguments
from src.model.model import MMEBModel
from src.model.processor import load_processor, QWEN2_VL, VLM_IMAGE_TOKENS, \
    Qwen2_VL_process_fn, LLAVA_QWEN2, FastVLM_process_fn

from PIL import Image
import numpy as np
import torch
import math
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F

from transformers.image_transforms import (
    convert_to_rgb,
    resize,
)

/workspace/VLM_Embed/vlm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/VLM_Embed/vlm/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/workspace/VLM_Embed/src/model/vlm_backbone/internvideo2/modeling_internvideo2.py:541: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
[2025-12-29 15:13:15,925] DEBUG [matplotlib:342] matplotlib data path: /workspace/VLM_Embed/vlm/lib/python3.12/site-packages/matplotlib/mpl-data
[2025-12-29 15:13:15,932] DEBUG [matplotlib:342] CONFIG

FusedMLP of flash_attn is not installed!!!
DropoutAddRMSNorm of flash_attn is not installed!!!
flash_attn_interface or bert_padding of flash_attn is not installed!!!


[2025-12-29 15:13:16,125] INFO [matplotlib.font_manager:1639] generated new fontManager
[2025-12-29 15:13:16,330] DEBUG [matplotlib.pyplot:496] Loaded backend module://matplotlib_inline.backend_inline version unknown.
[2025-12-29 15:13:16,333] DEBUG [matplotlib.pyplot:496] Loaded backend inline version unknown.


In [6]:
model_args = ModelArguments(
    model_name='apple/FastVLM-0.5B',
    # checkpoint_path='TIGER-Lab/VLM2Vec-Qwen2VL-2B',
    pooling='last',
    normalize=True,
    model_backbone=LLAVA_QWEN2,
    # lora=True
)
data_args = DataArguments()

processor = load_processor(model_args, None)
model = MMEBModel.build(model_args)
model = model.to('cuda', dtype=torch.bfloat16)
model.eval()

[2025-12-29 15:21:49,496] INFO [src.utils:21] Loading processor from: apple/FastVLM-0.5B


Processor load here for LLAVA-QWEN2


[2025-12-29 15:21:49,772] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
[2025-12-29 15:21:49,792] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/tokenizer_config.json HTTP/1.1" 200 0
[2025-12-29 15:21:50,628] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "GET /api/models/apple/FastVLM-0.5B/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64
[2025-12-29 15:21:50,931] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/vocab.json HTTP/1.1" 307 0
[2025-12-29 15:21:51,202] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/vocab.json HTTP/1.1" 200 0
[2025-12-29 15:21:51,205] DEBUG [filelock:331] Att

Detected model type: llava_qwen2
Determined model backbone: llava_qwen2


[2025-12-29 15:21:56,866] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/adapter_config.json HTTP/1.1" 404 0
[2025-12-29 15:21:57,140] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/model.safetensors HTTP/1.1" 302 0
[2025-12-29 15:21:57,144] DEBUG [filelock:331] Attempting to acquire lock 140189946268656 on /workspace/.hf_home/hub/.locks/models--apple--FastVLM-0.5B/4c70bf1ae23f362b9a623706bffbd0830999ea55c821c0b3545c49bd988095a7.lock
[2025-12-29 15:21:57,145] DEBUG [filelock:334] Lock 140189946268656 acquired on /workspace/.hf_home/hub/.locks/models--apple--FastVLM-0.5B/4c70bf1ae23f362b9a623706bffbd0830999ea55c821c0b3545c49bd988095a7.lock
[2025-12-29 15:21:57,883] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "GET /api/models/apple/FastVLM-0.5B/xet-read-token/16375720c2d673fa583e57e9876afde27549c7d0 HTTP/1.1" 200 365
[2025-12-29 15:22:09,960] DEBUG [filelock:364] 

MMEBModel(
  (encoder): LlavaQwen2ForCausalLM(
    (model): LlavaQwen2Model(
      (embed_tokens): Embedding(151936, 896)
      (layers): ModuleList(
        (0-23): 24 x Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        )
      )
   

In [19]:
processor.image_processor

CLIPImageProcessor {
  "crop_size": {
    "height": 1024,
    "width": 1024
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.0,
    0.0,
    0.0
  ],
  "image_processor_type": "CLIPImageProcessor",
  "image_std": [
    1.0,
    1.0,
    1.0
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 1024
  }
}

In [22]:
model.encoder

LlavaQwen2ForCausalLM(
  (model): LlavaQwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2